In [1]:
import sys
sys.path.append('../FaaSterConfig')

import FaaSterConfig
sys.argv = ['./FaaSterConfig.py', '../openFaas/stack.yml']

In [2]:
args, doe, stack = FaaSterConfig.main()

/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'GPU1' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]
/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'GPU2' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]
/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'GPU3' has dtype incompatible with float32, please 

In [3]:
doe

,CPU,Mem,NodeType,i,NodeTypeStr,funcName
0,0.5,248.0,0.0,1,GPU1,greeter_CPU0.5_Mem_248.0_GPU1
1,4.0,248.0,0.0,2,GPU1,greeter_CPU4.0_Mem_248.0_GPU1
2,16.0,248.0,0.0,3,GPU1,greeter_CPU16.0_Mem_248.0_GPU1
3,0.5,1024.0,0.0,4,GPU1,greeter_CPU0.5_Mem_1024.0_GPU1
4,4.0,1024.0,0.0,5,GPU1,greeter_CPU4.0_Mem_1024.0_GPU1
5,16.0,1024.0,0.0,6,GPU1,greeter_CPU16.0_Mem_1024.0_GPU1
6,0.5,16384.0,0.0,7,GPU1,greeter_CPU0.5_Mem_16384.0_GPU1
7,4.0,16384.0,0.0,8,GPU1,greeter_CPU4.0_Mem_16384.0_GPU1
8,16.0,16384.0,0.0,9,GPU1,greeter_CPU16.0_Mem_16384.0_GPU1
9,0.5,248.0,1.0,10,GPU2,greeter_CPU0.5_Mem_248.0_GPU2
